# General Overview of the Reconstruction Performances for a Moving Absorbing Boundary Condition

**Are we in SWAN?**

In [ ]:
%pip install --user crank-nicolson-numba

**Do we want the matplotlib interactive magicness?**

In [1]:
%matplotlib widget

**Library Import**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.integrate
from tqdm.notebook import tqdm
import crank_nicolson_numba.generic as cn
import itertools
# For parallelization
from joblib import Parallel, delayed

import nekhoroshev_tools as nt

## First, we need to define a realistic initial distribution

It's very simple and direct: if we decide to work in normalized emittance variables, and we work with a gaussian beam distribution with it's characteristic emittance, we have that
$$\rho_0(I) = e^{-I}$$
Where $I$, from now on, is expessed in sigma units.

In [3]:
def rho_0(I, damping_position=np.nan, l=np.nan):
    if np.isnan(damping_position) or np.isnan(l):
        return np.exp(-I)
    else:
        return np.exp(-I) / (1 + np.exp((I - damping_position)/l))

## Setting up a regular scenario

In [4]:
I_damping = 9.8
I_max = 5.0
I_star = 10.0
k = 0.33
exponent = 1/(2*k)

c = nt.standard_c(0.0, I_max, I_star, exponent)

In [5]:
cn_sampling = 50000
I_list, dI = np.linspace(0.0, I_max, cn_sampling, retstep=True)

cn_time_steps = 200
dt = nt.current_peak_time(I_damping, I_max, I_star, exponent, c)/cn_time_steps

# "Very fast" tracking

# "Long enough" tracking

## Working in the semi-stable region

Given $I_*=10.0$, we will firstly analyze the semi-stable region $[5.0-8.0]$ where we have the beginning of the saturation phase for $D(I)$.

In [6]:
steps_per_sample = 20

steps_per_iteration = 100
iterations = 20
macro_iterations = 5
long_track_steps = steps_per_iteration * iterations * (macro_iterations + 1)

## How's the regular current for a single, constant $I_a$?

In [ ]:
engine = cn.cn_generic(0, I_max, rho_0(I_list, I_damping, dI*5), dt, lambda x: nt.D(x, I_star, exponent, c, True))
data_0 = engine.get_data_with_x()

times_long, current_long = engine.current(long_track_steps, steps_per_sample, False)
data_long = engine.get_data_with_x()

/home/camontan/Insync/carlo.montanari3@studio.unibo.it/OneDrive_Biz/projects/new_games_with_diffusion/nekhoroshev_tools.py:37: RuntimeWarning: divide by zero encountered in double_scalars
  return c * np.exp(-2*np.power(I_star/I, exponent)) * (0.5 if halved else 1.0)
 19%|█▉        | 2254/12000 [01:30<05:44, 28.26it/s]

**That's how our distribution (slowly) evolves...**

(In order to see things it's necessary to work with matplotlib interactive mode and zoom a lot)

In [ ]:
plt.figure()
plt.plot(data_0[0], data_0[1], label="Initial condition")
plt.plot(data_long[0], data_long[1], label="After iterations")
plt.axvline(I_max, color="black", label="$I_a$")
plt.xlabel("$I$")
plt.ylabel("$\\rho$")
plt.legend()

**And here's the current!**

In [ ]:
plt.figure()
plt.plot(times_long, current_long)
plt.xlabel("$t$")
plt.ylabel("$J$")
plt.yscale("log")

## Experimenting with a bit of dancing!

In [ ]:
engine = cn.cn_generic(0, I_max, rho_0(I_list, I_damping, dI*5), dt, lambda x: nt.D(x, I_star, exponent, c, True))

t_absolute_out = []
t_relative_out = []
c_out = []
I_max_list = []
set_list = []

t_absolute_out_partial = []
t_relative_out_partial = []
c_out_partial = []
I_max_list_partial = []
set_list_partial = []

for j in tqdm(range(macro_iterations + 1)):
    if j != 0:
        engine.move_barrier_backward(0.2 * iterations)

    for i in tqdm(range(iterations)):
        time, current = engine.current(steps_per_iteration, steps_per_sample)
        if len(t_absolute_out) > 0:
            absolute_time = time + t_absolute_out[-1][-1]
        else:
            absolute_time = time.copy()
        t_relative_out.append(time)
        t_absolute_out.append(absolute_time)
        c_out.append(current)
        I_max_list.append(engine.I_max)
        set_list.append(i)
        if i != 0:
            t_relative_out_partial.append(time)
            t_absolute_out_partial.append(absolute_time)
            c_out_partial.append(current)
            I_max_list_partial.append(engine.I_max)
            set_list_partial.append(i)
        engine.move_barrier_forward(0.2)

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
for i in range(len(t_absolute_out)):
    ax2.plot(
        [t_absolute_out[i][0],t_absolute_out[i][-1], t_absolute_out[i][-1]],
        [I_max_list[i],I_max_list[i],(I_max_list[i+1] if i+1 != len(I_max_list) else I_max_list[i])],
        c="black", alpha=0.5
    )
    ax1.plot(t_absolute_out[i], c_out[i])
    if i != len(t_absolute_out) - 1:
        ax1.plot(
            [t_absolute_out[i][-1], t_absolute_out[i][-1]],
            [c_out[i][-1], c_out[i+1][0]],
            c="black"
        )

ax1.plot(times_long, current_long, c="C0", linestyle="dashed")

ax1.set_yscale("log")
ax1.set_ylabel("Current")
ax1.set_xlabel("time")
ax2.set_ylabel("Barrier position")

In [ ]:
fig, ax1 = plt.subplots()

slice_size = len(current_long) // len(t_absolute_out)

ax2 = ax1.twinx()
for i in range(len(t_absolute_out)):
    ax2.plot(
        [t_absolute_out[i][0],t_absolute_out[i][-1], t_absolute_out[i][-1]],
        [I_max_list[i],I_max_list[i],(I_max_list[i+1] if i+1 != len(I_max_list) else I_max_list[i])],
        c="black", alpha=0.5
    )
    ax1.plot(t_absolute_out[i], c_out[i]/(current_long[slice_size * i:slice_size * (i+1)]))
    if i != len(t_absolute_out) - 1:
        ax1.plot(
            [t_absolute_out[i][-1], t_absolute_out[i][-1]],
            np.array([c_out[i][-1], c_out[i+1][0]])/current_long[slice_size*(i+1)],
            c="black"
        )

#ax1.plot(times_long, current_long, c="C0", linestyle="dashed")

#ax1.set_yscale("log")
ax1.set_ylabel("`Normalized` Current")
ax1.set_xlabel("time")
ax2.set_ylabel("Barrier position")

In [ ]:
plt.figure()
plt.scatter(
    I_max_list_partial,
    [t_absolute_out_partial[i][np.argmax(c_out_partial[i])] - (t_absolute_out[i-1][-1] if i != 0 else 0) for i in range(len(t_absolute_out_partial))]
)